In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv


In [2]:
df = pd.read_csv('/kaggle/input/womens-ecommerce-clothing-reviews/Womens Clothing E-Commerce Reviews.csv')
df.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [3]:
data = df['Review Text']
data = data * 10 
data 

0        Absolutely wonderful - silky and sexy and comf...
1        Love this dress!  it's sooo pretty.  i happene...
2        I had such high hopes for this dress and reall...
3        I love, love, love this jumpsuit. it's fun, fl...
4        This shirt is very flattering to all due to th...
                               ...                        
23481    I was very happy to snag this dress at such a ...
23482    It reminds me of maternity clothes. soft, stre...
23483    This fit well, but the top was very see throug...
23484    I bought this dress for a wedding i have this ...
23485    This dress in a lovely platinum is feminine an...
Name: Review Text, Length: 23486, dtype: object

In [4]:
from six.moves import reduce 
from multiprocessing import Pool 
pool = Pool(4) 
from datetime import datetime 

import sklearn
from sklearn.feature_extraction import text 
from collections import Counter 
import re 

## Without using Multi-processing Library 

In [5]:
def clean_word(word):
    return re.sub(r'[^\w\s]', '', word).lower() 

def word_not_in_stopwords(word):
    return word not in sklearn.feature_extraction.text.ENGLISH_STOP_WORDS and word and word.isalpha()

def find_top_words(data):
    cnt = Counter() 
    for text in data:
        tokens_in_text = str(text).split() 
        tokens_in_text = map(clean_word, tokens_in_text)
        tokens_in_text = filter(word_not_in_stopwords, tokens_in_text)
        cnt.update(tokens_in_text)

    return cnt.most_common(10)

%time find_top_words(data)

CPU times: user 32.2 s, sys: 166 ms, total: 32.4 s
Wall time: 32.4 s


[('dress', 101792),
 ('size', 84539),
 ('love', 78729),
 ('fit', 70796),
 ('like', 69002),
 ('wear', 63312),
 ('great', 56476),
 ('im', 55564),
 ('just', 54406),
 ('fabric', 46965)]

In [6]:
import multiprocessing as np

def mapper(text):
    tokens_in_text = str(text).split() 
    tokens_in_text = map(clean_word, tokens_in_text)
    tokens_in_text = filter(word_not_in_stopwords, tokens_in_text)
    
    return Counter(tokens_in_text)
   
def reducer(cnt1, cnt2):
    cnt1.update(cnt2)
    return cnt1
    
def chunk_mapper(chunk):
    mapped = map(mapper, chunk) 
    reduced = reduce(reducer, mapped) 
    return reduced
    
# running these functions 
def chunks(l, chunk_size):
    result = []
    chunk = []
    for item in l:
        chunk.append(item)
        if len(chunk) == chunk_size:
            result.append(chunk) 
            chunk = []
    return result 

## Common Keywords in Woman's Clothing review using Map Reduce 

In [7]:
%%time
data_chunks = chunks(data, 4)

def main():
    pool = np.Pool(4)
    start = datetime.now() 
    mapped = pool.map(chunk_mapper, data_chunks) 
    reduced = reduce(reducer, mapped) 
    end = datetime.now() 
    t = end-start 
    print(start, end) 
    print(t.seconds)
    print(reduced.most_common(10))
    
if __name__ == "__main__":
    main()

2020-09-14 10:25:17.040617 2020-09-14 10:25:32.210293
15
[('dress', 101762), ('size', 84539), ('love', 78729), ('fit', 70786), ('like', 68992), ('wear', 63302), ('great', 56476), ('im', 55564), ('just', 54396), ('fabric', 46965)]
CPU times: user 616 ms, sys: 232 ms, total: 848 ms
Wall time: 15.2 s
